In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
from sklearn import *

In [ ]:
df_train=pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/train.csv")
df_train.head()

In [ ]:
def clear_tweets(text):
    text=re.sub('<[^>]*>','',text)
    emojis=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub('[\W]+',' ',text.lower()) +' '.join(emojis).replace('-','')

    return text   

In [ ]:
df_train['tweet']=df_train['tweet'].apply(clear_tweets)
df_train.head()

In [ ]:
df_train['label'].unique()

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop=stopwords.words('english')

stop_words = set(stopwords.words("english"))


In [ ]:
stemmer = PorterStemmer()

In [ ]:
def func_2(tweets):
    token=word_tokenize(tweets)
    filtered_list = []
    for word in token:
        if word.casefold() not in stop_words:
            filtered_list.append(word)
    stemmed_words = [stemmer.stem(word) for word in filtered_list]
    return stemmed_words

In [ ]:
import nltk
nltk.download('punkt')
new_list=[]
arr=[]
i=0
for tweet in df_train['tweet']:
    new_list=func_2(tweet)
    joined_str=""
    for word in new_list:
        joined_str=joined_str+" "+word
    arr.append(joined_str)

In [ ]:
df_train['new_tweet']=arr

In [ ]:
df_train

In [ ]:
X_neg=[]
X_pos=[]
Y_pos=[]
Y_neg=[]
for t in df_train['new_tweet'][df_train['label']==0]:
    X_neg.append(t)
    
for t in df_train['label'][df_train['label']==0]:
    Y_neg.append(t)

for t in df_train['new_tweet'][df_train['label']==1]:
    X_pos.append(t)
    
for t in df_train['label'][df_train['label']==1]:
    Y_pos.append(t)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000)

X_vec=vectorizer.fit_transform(df_train.new_tweet)

In [ ]:
y=df_train.label.values
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_vec,y,random_state=1,test_size=0.25,shuffle=False)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
df_test=pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/test.csv")
test=df_test['tweet']
test=test.apply(clear_tweets)
test=test.apply(func_2)



arr=[]

for tweet in test:

    joined_str=""
    for word in tweet:
        joined_str=joined_str+" "+word
    arr.append(joined_str)

test=pd.DataFrame(arr,columns=['tweet'])


test.head()


In [ ]:
xtest=vectorizer.fit_transform(test.tweet)
xtest.shape
ytest=clf.predict(xtest)


In [ ]:
final=pd.DataFrame(df_test['tweet'],columns=['tweet'])
final['label']=ytest
final.head()

In [ ]:
data=['Congratulations to His Excellency Ebrahim Raisi on his election as President of the Islamic Republic of Iran',
      ' I look forward to working with him to further strengthen the warm ties between India and Iran.',
      'Baghor shrine is a modest temple that only has a stone platform under Neem tree and a triangular shaped Yantra worshiped as Goddess in the middle of platform.',
      'Yet, it has been worshiped continuously since 11,000 years from Paleolithic hunter gatherers to modern day inhabitants',
      'This video legitimately made me wanna try Takis, and then the comment section talked me completely out of it.',
      'i am living',
      'guys for some reason i am in a minecraft tournament i am gonna be streaming it in like an hour',
      'astronaut in the ocean was written and performed by a youth pastor i’m sure of it']

In [ ]:
data=pd.DataFrame(data,columns=['tweet'])
data.head()


In [ ]:

text=data['tweet'].apply(clear_tweets)
text=text.apply(func_2)

arr=[]
for tweet in text:
    joined_str=""
    for word in tweet:
        joined_str=joined_str+" "+word
    arr.append(joined_str)
    
text=pd.DataFrame(arr,columns=['tweet'])
test.shape

test=test.append(text,ignore_index=True)
test.tail(10)



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec= TfidfVectorizer(max_features=10000)  

from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
xval=vec.fit_transform(test.tweet)

yval=clf.predict(xval)

final=pd.DataFrame(test['tweet'],columns=['tweet'])
final['label']=yval
final.head()

In [ ]:
(final['label']).value_counts()
final.head()

In [ ]:
final.tail(20)

In [ ]:
final['label'].value_counts()

In [ ]:
tweet=['You are beautiful love ']
data=pd.DataFrame(tweet,columns=['tweet'])
data.head()
text=data['tweet'].apply(clear_tweets)
text=text.apply(func_2)

arr=[]
for tweet in text:
    joined_str=""
    for word in tweet:
        joined_str=joined_str+" "+word
    arr.append(joined_str)
    
text=pd.DataFrame(arr,columns=['tweet'])
# test.shape

test=test.append(text,ignore_index=True)
# test.tail(10)
vec= TfidfVectorizer(max_features=10000)
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
xval=vec.fit_transform(test.tweet)

yval=clf.predict(xval)

final=pd.DataFrame(test['tweet'],columns=['tweet'])
final['label']=yval
final.head()


In [ ]:
final['label'].tail(1)

In [ ]:
import pickle
